In [ ]:
import pandas as pd
import numpy as np
import sklearn
import category_encoders as ce
from sklearn.impute import KNNImputer
import math
import matplotlib.pyplot as plt
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

In [ ]:
allegro = pd.read_csv("allegro-api-transactions.csv")


In [ ]:
allegro.head()

# Target encoding na allegro_data

In [ ]:
allegro_target_encoding = allegro.copy(deep = True)
means = allegro.groupby("it_location")['price'].mean()
allegro_target_encoding['it_location'] = allegro_target_encoding['it_location'].map(means)
allegro_target_encoding.head()

W przypadku onehot encodingu, ze względu na to że mamy bardzo dużo wartości w kolumnie 'it_location', pojawi się bardzo dużo nowych kolumn. Może okazać się, że będziemy mieli za dużą ilość kolumn żeby wytrenować nasz model. One hot encoding będzie dobrym rozwiązaniem jeśli tabela kategoryczna, która przekształcamy ma mało różnych wartości.

# One hot encoding

In [ ]:
allegro_one_hot = allegro.copy(deep = True)
main_categories = pd.get_dummies(allegro['main_category'])
allegro_one_hot = pd.concat([allegro_one_hot, main_categories], axis=1)
allegro_one_hot.drop(labels = ['main_category'], axis = 1,inplace = True)
allegro_one_hot

Metoda one hot encoding tworzy nam z kolumny o wyrazach kategorycznych macierz ( n x m ), gdzie n - liczba wierszy, m - liczba kategorii w kolumnie. Każdy wiersz ma jedynkę w tej kolumnie z której miał wartośc i zera w innych kolumnach. Dobra metoda gdy mamy mało kategorii.

In [ ]:
allegro_cat = allegro.copy(deep = True)
CatBoostEncoder = ce.CatBoostEncoder()
CatBoostEncoder = CatBoostEncoder.fit(allegro['main_category'], y = allegro['price'])
allegro_cat['main_category'] = CatBoostEncoder.transform(allegro_cat["main_category"])
allegro_cat

Metoda działa tak samo jak target encoding z tą różnicą, że pomija wartość targetu aktualnego wiersza, aby zminimalizować efekt outlierów. Metoda działa w locie.

In [ ]:
allegro_count = allegro.copy(deep = True)
CountEncoder = ce.CountEncoder()
CountEncoder = CountEncoder.fit(allegro_count['main_category'])
allegro_count['main_category'] = CountEncoder.transform(allegro_count["main_category"])
allegro_count

Metoda zamienia każdą wartość na ilość wystąpień danej kategorii w kolumnie.

In [ ]:
allegro_reduced = allegro[['price','it_seller_rating', 'it_quantity']]
allegro_reduced = allegro_reduced[0:40000]
n = len(allegro_reduced['price'])
allegro_reduced

Zmniejszylem liczbe rekordow bo bardzo wolno mi dzialal KKNIImputer :(

In [ ]:
results = [0 for i in range(10)]
for i in range(10):
    X = allegro_reduced.copy(deep=True)
    # zmieniam ziarno za kazdym razem zeby miec rozne wiersze w roznich eksperymentach
    np.random.seed((i+1) * 10)
    chosen_idx = np.random.choice(n , replace = False, size = 4000)
    X.it_seller_rating[chosen_idx] = np.nan
    imputer = KNNImputer(n_neighbors=5, weights="uniform")
    X = imputer.fit_transform(X)
    results[i] = math.sqrt(np.sum(pow(X[:,1] - allegro_reduced["it_seller_rating"], 2)) / n)
    

In [ ]:
results 

In [ ]:
x = [i for i in range(10)]
plt.scatter(x,results)
plt.show()

In [ ]:
standard_deviation = math.sqrt(np.sum(pow(results - results.mean(),2)))
standard_deviation

To odchylenie wyszlo bardzo duże i wgl jakoś kiepsko to wygląda

In [ ]:
second_results = [0 for i in range(10)]
for i in range(10):
    X = allegro_reduced.copy(deep=True)
    # zmieniam ziarno za kazdym razem zeby miec rozne wiersze w roznich eksperymentach
    np.random.seed((i+1) * 10)
    chosen_idx = np.random.choice(n , replace = False, size = 4000)
    X.it_seller_rating[chosen_idx] = np.nan
    X.it_quantity[chosen_idx] = np.nan
    imputer = KNNImputer(n_neighbors=5, weights="uniform")
    X = imputer.fit_transform(X)
    second_results[i] = math.sqrt(np.sum(pow(X[:,1] - allegro_reduced["it_seller_rating"], 2)) / n)

In [ ]:
second_results

In [ ]:
second_standard_deviation = math.sqrt(np.sum(pow(second_results - results.mean(),2)))
second_standard_deviation

In [ ]:
x = [i for i in range(10)]
plt.scatter(x,second_results)
plt.show()

Po usunięciu wartości z obu kolumn różnice w wynikach pomiędzy prawdziwą kolumną it_seller_rating, a to stworzoną po algorytmie k-nearest-neighbours są bardzo duże, również odchylenie standardowe jest znacznie większe. Wydaje mi się, że w obydwu przypadkach błąd jest stosunkowo duży.